In [4]:
import pandas as pd
import numpy as np

In [5]:
#chessData = pd.read_csv('Data/chessData.csv')
#random_evals = pd.read_csv('Data/random_evals.csv')-
tactic_evals = pd.read_csv('Data/tactic_evals.csv')

In [6]:
tactic_evals = tactic_evals.dropna()
tactic_evals['isCheck'] = tactic_evals['Evaluation'].str.contains("#")
tactic_evals['isCheck'] = list(map(int, tactic_evals['isCheck']))

#predictors_raw = tactic_evals_check.drop(['Evaluation', 'isCheck'], axis=1)
#predictors_raw.head()

In [13]:
for move in tactic_evals['Move']:
    if len(move) != 4:
        print(move)

d2e1q
c2c1q
d7d8q
f7e8r
f2f1q
a7a8q
f7g8q
a7a8q
a7b8q
d7d8q
h2h1q
c7d8q
d7d8q
g2g1q
d2d1q
e7e8q
d2d1q
c7c8r
b2b1q
h2h1q
d7d8q
c2c1q
d7e8q
d2d1q
g7g8q
d2d1q
b7b8q
e7e8q
b2b1q
c7c8q
f2f1q
a2b1q
f2g1b
f7g8q
d7d8q
e7e8q
b7b8q
h2h1q
e2e1q
a2a1q
d7d8q
b2b1q
e7e8q
a7a8q
f2f1q
g7g8q
c2d1q
d2d1q
a7a8q
b2b1q
d7d8q
c2c1q
f2f1q
b2b1q
h2h1q
b7b8q
d7d8q
d7d8q
h7h8q
c7c8q
f2f1q
c2c1q
e7f8q
f7f8q
e2e1q
h2h1q
d7d8q
d7c8q
h7h8q
b2a1q
h2h1r
h2h1q
f7f8b
d7d8q
b7b8q
h2h1q
f7f8q
h2h1q
a7a8q
h7h8q
h7h8r
b7b8q
d2d1q
a7a8q
h7h8q
g7g8q
c7c8q
h7h8q
a2a1q
d7d8q
a2a1q
f7f8q
a2a1r
a7a8q
e2e1q
h7h8q
d7e8q
c2c1q
h7h8q
d2e1q
e2d1r
b2b1q
d2e1q
c7d8r
e2e1q
f2f1q
d7d8q
h2h1q
a7a8q
h7h8q
c2c1q
g2g1q
d7e8q
c7d8q
e7f8q
c2c1q
g7g8q
h2h1q
h2h1q
c7c8q
d2d1q
f2f1q
h7h8q
h7h8q
e7e8q
d2d1q
a7a8q
d2e1q
c2c1q
e7e8q
b2b1q
h7h8q
d2d1q
h2h1q
g2g1n
e2e1q
g7g8q
h7h8q
e2e1q
c7d8q
d2d1q
d7e8q
b7a8q
d7d8q
a2a1n
g7g8q
d7d8q
c2c1q
e2e1q
e7f8q
a2b1q
c2c1q
c2b1q
b7b8q
c7c8q
c2c1r
a7a8q
d2d1q
d2d1q
f7g8q
b2b1q
f7f8q
h7h8q
d7d8q
b2b1q
c2c1q
g7g8

## Pré-processamento dos dados ##

Formato de um código FEN: rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1

Para utilizar esses dados em um modelo de IA, precisamos fazer algums tratamentos para transformar eles em um input

#### Pré-processamento código FEN ####

Tokenização: Podemos análisar o código mais facilmente tokenizando o valor para os valores presentes em suas linhas

Remoção de dados irrelevantes: Para nosso objetivo, que não envolve validação de jogadas, as informações após o tabuleiro não são úteis

Converter números para 0: Transformar um número naquela quantidade de 0's é importante para representar as casas vazias

Conversão para matriz: Utilizando a pontuação de sarrat para converter peças em números e transformar o código em uma matriz 8x8


#### Pré-processamento jogada ####

Tokenização: Similarmente a como fizemos com a notação FEN, para as jogadas, iremos separar os val



In [1]:
sarrat = {
    'P':1,
    'N':3.1,
    'B':3.3,
    'R':5,
    'Q':7.9,
    'K':2.2,
    'p':-1,
    'n':-3.1,
    'b':-3.3,
    'r':-5,
    'q':-7.9,
    'k':-2.2
}

In [3]:
import re
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer

In [ ]:
def separate_board(board):
    return board.replace('/', ' ').split()

def remove_extras(board):
    return board[0:8]

def num_to_zeros(board):
    for i in range(len(board)):
        board[i] = re.sub(r'[1-9]', '0', board[i])
    return board
            
def fen_to_matrix(fen):
    matrix = []
    for line in fen:
        row = []
        for letter in line:
            row.append(sarrat.get(f'{letter}', letter))
        matrix.append(row)
    return matrix

def separate_move(move):
    separated_move = []
    for i in range(len(move)):
        separated_move.append(move[i])
    return separated_move


In [ ]:
pipeline_transform_fen = Pipeline([
    ('Separate board into lines', FunctionTransformer(separate_board)),
    ('Remove no-board info', FunctionTransformer(remove_extras)),
    ('Convert num to zeros', FunctionTransformer(num_to_zeros)),
    ('Convert fen notation to matrix', FunctionTransformer(fen_to_matrix))
])

In [ ]:
from tensorflow.keras.layers import Dense, Flatten, MaxPooling2D, Concatenate, Input, Conv2D
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping